## Support Files

In [0]:
!wget https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/16188500/rumoureval2019.tar.bz2
!tar xf rumoureval2019.tar.bz2

--2019-11-11 10:36:01--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/16188500/rumoureval2019.tar.bz2
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.105.146
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.105.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16512097 (16M) [application/octet-stream]
Saving to: ‘rumoureval2019.tar.bz2.1’

rumoureval2019.tar. 100%[===================>]  15.75M  10.2MB/s    in 1.5s    

2019-11-11 10:36:04 (10.2 MB/s) - ‘rumoureval2019.tar.bz2.1’ saved [16512097/16512097]



KeyboardInterrupt: ignored

In [0]:
!unzip rumoureval2019/rumoureval-2019-test-data.zip
!unzip rumoureval2019/rumoureval-2019-training-data.zip

Archive:  rumoureval2019/rumoureval-2019-test-data.zip
   creating: rumoureval-2019-test-data/
   creating: rumoureval-2019-test-data/twitter-en-test-data/
   creating: rumoureval-2019-test-data/twitter-en-test-data/barbara-bush/
   creating: rumoureval-2019-test-data/twitter-en-test-data/barbara-bush/986801053168488449/
  inflating: rumoureval-2019-test-data/twitter-en-test-data/barbara-bush/986801053168488449/structure.json  
   creating: rumoureval-2019-test-data/twitter-en-test-data/barbara-bush/986801053168488449/source-tweet/
  inflating: rumoureval-2019-test-data/twitter-en-test-data/barbara-bush/986801053168488449/source-tweet/986801053168488449.json  
   creating: rumoureval-2019-test-data/twitter-en-test-data/barbara-bush/986801053168488449/replies/
  inflating: rumoureval-2019-test-data/twitter-en-test-data/barbara-bush/986801053168488449/replies/986803257409499136.json  
  inflating: rumoureval-2019-test-data/twitter-en-test-data/barbara-bush/986801053168488449/replies/9868

## Support Functions

In [0]:
TRAIN_DATA_PREFIX = "rumoureval-2019-training-data"

from tree2branches import tree2branches
def load_true_labels():
    tweet_label_dict = {}
    veracity_label_dict = {}
    path_dev = os.path.join(TRAIN_DATA_PREFIX, "dev-key.json")
    with open(path_dev, 'r') as f:
        dev_key = json.load(f)

    path_train = os.path.join(TRAIN_DATA_PREFIX, "train-key.json")
    with open(path_train, 'r') as f:
        train_key = json.load(f)

    tweet_label_dict['dev'] = dev_key['subtaskaenglish']
    tweet_label_dict['train'] = train_key['subtaskaenglish']

    veracity_label_dict['dev'] = dev_key['subtaskbenglish']
    veracity_label_dict['train'] = train_key['subtaskbenglish']

    return tweet_label_dict, veracity_label_dict

In [0]:
def load_test_data_twitter(set_path ="rumoureval-2019-test-data/twitter-en-test-data"):
    allconv = []
    train_dev_split = {}
    train_dev_split['dev'] = []
    train_dev_split['train'] = []
    train_dev_split['test'] = []
    tweet_data = sorted(os.listdir(set_path))
    newfolds = [i for i in tweet_data if i[0] != '.']
    tweet_data = newfolds # conversation ids, source post id == conversation id
    conversation = {}
    # build conversations for tweet group

    for tweet_topic in tweet_data:
        path = os.path.join(set_path,tweet_topic)
        tweet_topic_data = sorted(os.listdir(path))
        tweet_topic_data = [i for i in tweet_topic_data if i[0] != '.']
        for foldr in tweet_topic_data:
            flag = 0
            conversation['id'] = foldr
            tweets = []
            path_repl = path + '/' + foldr + '/replies'
            files_t = sorted(os.listdir(path_repl))
            newfolds = [i for i in files_t if i[0] != '.']
            files_t = newfolds
            flag = "test"
            if files_t != []:

                # iterate over json reply files
                for repl_file in files_t:
                    with open(os.path.join(path_repl, repl_file)) as f:
                        for line in f:
                            tw = json.loads(line)
                            tw['used'] = 0

                            tw['set'] = flag
                            tweets.append(tw)
                            if tw['text'] is None:
                                print("Tweet has no text", tw['id'])
                conversation['replies'] = tweets

                path_src = path + '/' + foldr + '/source-tweet'
                files_t = sorted(os.listdir(path_src))
                with open(os.path.join(path_src, files_t[0])) as f:
                    for line in f:
                        src = json.loads(line)
                        src['used'] = 0
                        scrcid = src['id_str']
                        src['set'] = flag

                conversation['source'] = src
                if src['text'] is None:
                    print("Tweet has no text", src['id'])
                path_struct = path + '/' + foldr + '/structure.json'
                with open(path_struct) as f:
                    for line in f:
                        struct = json.loads(line)
                if len(struct) > 1:
                    new_struct = {}
                    new_struct[foldr] = struct[foldr]
                    struct = new_struct
                    # Take item from structure if key is same as source tweet id
                conversation['structure'] = struct

                branches = tree2branches(conversation['structure'])
                conversation['branches'] = branches
                train_dev_split[flag].append(conversation.copy())
                allconv.append(conversation.copy())

            # if no replies are present, still add just source
            else:
                flag = 'test'
                path_src = path + '/' + foldr + '/source-tweet'
                files_t = sorted(os.listdir(path_src))
                with open(os.path.join(path_src, files_t[0])) as f:
                    for line in f:
                        src = json.loads(line)
                        src['used'] = 0
                        scrcid = src['id_str']
                        src['set'] = flag

                conversation['source'] = src
                if src['text'] is None:
                    print("Tweet has no text", src['id'])

                path_struct = path + '/' + foldr + '/structure.json'
                with open(path_struct) as f:
                    for line in f:
                        struct = json.loads(line)
                if len(struct) > 1:
                    # print "Structure has more than one root"
                    new_struct = {}
                    new_struct[foldr] = struct[foldr]
                    struct = new_struct
                    # Take item from structure if key is same as source tweet id
                conversation['structure'] = struct
                branches = tree2branches(conversation['structure'])

                conversation['branches'] = branches
                train_dev_split[flag].append(conversation.copy())
                allconv.append(conversation.copy())
                print(type(conversation['branches']))

                print(foldr)

    return train_dev_split

# %%
def load_dataset():
    # Load labels and split for task A and task B
    tweet_label_dict, veracity_label_dict = load_true_labels()
    dev = tweet_label_dict['dev']
    train = tweet_label_dict['train']
    dev_tweets = dev.keys()
    train_tweets = train.keys()
    # Load folds and conversations
    path_to_folds = os.path.join(TRAIN_DATA_PREFIX, 'twitter-english')
    folds = sorted(os.listdir(path_to_folds))
    newfolds = [i for i in folds if i[0] != '.']
    folds = newfolds
    cvfolds = {}
    allconv = []
    train_dev_split = {}
    train_dev_split['dev'] = []
    train_dev_split['train'] = []
    train_dev_split['test'] = []
    # iterate over all tweet groups [reffered to as 'folds'] - charliehebdo etc...
    for nfold, fold in enumerate(folds):
        path_to_tweets = os.path.join(path_to_folds, fold)
        tweet_data = sorted(os.listdir(path_to_tweets))
        newfolds = [i for i in tweet_data if i[0] != '.']
        tweet_data = newfolds # conversation ids, source post id == conversation id
        conversation = {}
        # build conversations for tweet group
        for foldr in tweet_data:
            flag = 0
            conversation['id'] = foldr
            tweets = []
            path_repl = path_to_tweets + '/' + foldr + '/replies'
            files_t = sorted(os.listdir(path_repl))
            newfolds = [i for i in files_t if i[0] != '.']
            files_t = newfolds
            if files_t != []:
                # iterate over json reply files
                for repl_file in files_t:
                    with open(os.path.join(path_repl, repl_file)) as f:
                        for line in f:
                            tw = json.loads(line)
                            tw['used'] = 0
                            replyid = tw['id_str']

                            # check if tweet belongs to dev fold
                            if replyid in dev_tweets:
                                tw['set'] = 'dev'
                                tw['label'] = dev[replyid]
                                #                        train_dev_tweets['dev'].append(tw)
                                if flag == 'train':
                                    print("The tree is split between sets", foldr)
                                flag = 'dev'
                            elif replyid in train_tweets:
                                tw['set'] = 'train'
                                tw['label'] = train[replyid]
                                #                        train_dev_tweets['train'].append(tw)
                                if flag == 'dev':
                                    print("The tree is split between sets", foldr)
                                flag = 'train'
                            else:
                                print("Tweet was not found! ID: ", foldr)
                            tweets.append(tw)
                            if tw['text'] is None:
                                print("Tweet has no text", tw['id'])
                conversation['replies'] = tweets

                path_src = path_to_tweets + '/' + foldr + '/source-tweet'
                files_t = sorted(os.listdir(path_src))
                with open(os.path.join(path_src, files_t[0])) as f:
                    for line in f:
                        src = json.loads(line)
                        src['used'] = 0
                        scrcid = src['id_str']
                        src['set'] = flag
                        src['label'] = tweet_label_dict[flag][scrcid]

                conversation['source'] = src
                conversation['veracity'] = veracity_label_dict[flag][scrcid]
                if src['text'] is None:
                    print("Tweet has no text", src['id'])
                path_struct = path_to_tweets + '/' + foldr + '/structure.json'
                with open(path_struct) as f:
                    for line in f:
                        struct = json.loads(line)
                if len(struct) > 1:
                    # I had to alter the structure of this conversation
                    if foldr == '553480082996879360':
                        new_struct = {}
                        new_struct[foldr] = struct[foldr]
                        new_struct[foldr]['553495625527209985'] = struct['553485679129534464']['553495625527209985']
                        new_struct[foldr]['553495937432432640'] = struct['553490097623269376']['553495937432432640']
                        struct = new_struct
                    else:
                        new_struct = {}
                        new_struct[foldr] = struct[foldr]
                        struct = new_struct
                    # Take item from structure if key is same as source tweet id
                conversation['structure'] = struct

                branches = tree2branches(conversation['structure'])
                conversation['branches'] = branches
                train_dev_split[flag].append(conversation.copy())
                allconv.append(conversation.copy())
            else:
                flag = 'train'
                path_src = path_to_tweets + '/' + foldr + '/source-tweet'
                files_t = sorted(os.listdir(path_src))
                with open(os.path.join(path_src, files_t[0])) as f:
                    for line in f:
                        src = json.loads(line)
                        src['used'] = 0
                        scrcid = src['id_str']
                        src['set'] = flag
                        src['label'] = tweet_label_dict[flag][scrcid]

                conversation['source'] = src
                conversation['veracity'] = veracity_label_dict[flag][scrcid]
                if src['text'] is None:
                    print("Tweet has no text", src['id'])

                path_struct = path_to_tweets + '/' + foldr + '/structure.json'
                with open(path_struct) as f:
                    for line in f:
                        struct = json.loads(line)
                if len(struct) > 1:
                    # print "Structure has more than one root"
                    new_struct = {}
                    new_struct[foldr] = struct[foldr]
                    struct = new_struct
                    # Take item from structure if key is same as source tweet id
                conversation['structure'] = struct
                branches = tree2branches(conversation['structure'])

                conversation['branches'] = branches
                train_dev_split[flag].append(conversation.copy())
                allconv.append(conversation.copy())

                print(foldr)

        cvfolds[fold] = allconv
        allconv = []

    return train_dev_split

In [0]:
import os
import json
from pprint import pprint
x=load_dataset()
print(len(x['train']),len(x['dev']))
import pandas as pd
print(x['train'][0]['source']['text'])
print(x.keys(),x['train'][0].keys(),x['train'][0]['replies'][0].keys())
x_tot=x['train']+x['dev']
print(len(x_tot))
# pprint(x['train'][0])

580323060533764097
297 28
France: 10 people dead after shooting at HQ of satirical weekly newspaper #CharlieHebdo, according to witnesses http://t.co/FkYxGmuS58
dict_keys(['dev', 'train', 'test']) dict_keys(['id', 'replies', 'source', 'veracity', 'structure', 'branches']) dict_keys(['contributors', 'truncated', 'text', 'in_reply_to_status_id', 'id', 'favorite_count', 'source', 'retweeted', 'coordinates', 'entities', 'in_reply_to_screen_name', 'id_str', 'retweet_count', 'in_reply_to_user_id', 'favorited', 'user', 'geo', 'in_reply_to_user_id_str', 'lang', 'created_at', 'in_reply_to_status_id_str', 'place', 'used', 'set', 'label'])
325


In [0]:
new_feature_keys = ['upvote_ratio', 'followers_count', 'is_verified', 'has_media']

## Test Data Prep

In [0]:
text_x = load_test_data_twitter()

In [0]:
x_test = text_x['test']

In [0]:
task_a_out = []
with open('output.txt') as f:
  for line in f.readlines():
    task_a_out.append(line.strip())

In [0]:
task_a_out = list(map(int,task_a_out))

In [0]:
source_ids = []
for i in range(len(x_test)):
  source_ids.append(x_test[i]['id'])

In [0]:
ctr = 0
upvote = []
for i in range(len(x_test)):
  source_res = []
  for j in range(len(x_test[i]['replies'])):
    source_res.append(task_a_out[ctr])
    ctr+=1
  upvote.append(source_res)

In [0]:
upvote_ratio = []
for item in upvote:
  support = item.count(3)
  upvote_ratio.append(support/len(item))

In [0]:
df = pd.DataFrame(columns=['source'])
count1=0
for i in range(len(x_test)):
    df.loc[count1] = [x_test[i]['source']['text']]
    count1=count1+1
print(df.head())

                                              source
0  #Breaking: Pentagon releases video of the “mot...
1  Rep. Sheila Jackson Lee has no shame. I still ...
2  ICYMI: "Rep Sheila Jackson Lee (D-Tx) Wants Hu...
3  In 2003, Houston U.S. Rep. Sheila Jackson Lee ...
4  Noah’s Ark Theme Park Destroyed in a Flood? ht...


In [0]:
def _finditem(obj, key):
    if key in obj: return obj[key]
    for k, v in obj.items():
        if isinstance(v,dict):
            item = _finditem(v, key)
            if item is not None:
                return item

In [0]:
is_verified = []
for item in x_test:
  if _finditem(item,'verified'):
    is_verified.append(1)
  else:
    is_verified.append(0)

In [0]:
has_media = []
for item in x_test:
  if (_finditem(item,'media')) is not None:
    has_media.append(1)
  else:
    has_media.append(0)

In [0]:
followers_count = []
for item in x_test:
  followers_count.append(_finditem(item,'followers_count'))

In [0]:
extra_features_df = pd.DataFrame(
    {
      'upvote_ratio':upvote_ratio,
      'followers_count':followers_count,
      'is_verified':is_verified,
      'has_media':has_media
       
    })

In [0]:
final_test_df= pd.DataFrame(
    {
      'source':df['source'].values,
      'upvote_ratio':upvote_ratio,
      'followers_count':followers_count,
      'is_verified':is_verified,
      'has_media':has_media
    })

In [0]:
final_test_df.head()

,source,upvote_ratio,followers_count,is_verified,has_media
0,#Breaking: Pentagon releases video of the “mot...,0.038462,4280227,1,1
1,Rep. Sheila Jackson Lee has no shame. I still ...,0.000000,7438,0,0
2,"ICYMI: ""Rep Sheila Jackson Lee (D-Tx) Wants Hu...",0.052632,446415,1,1
3,"In 2003, Houston U.S. Rep. Sheila Jackson Lee ...",0.285714,4727,0,0
4,Noah’s Ark Theme Park Destroyed in a Flood? ht...,0.000000,215512,1,0


In [0]:
import json

In [0]:
with open('test.json') as f:
  data = json.load(f)

In [0]:
data = data['subtaskbenglish']

In [0]:
test_y = []
for id in source_ids:
  test_y.append(data[id])

In [0]:
def convert_test_veracity(vals):
  res = []
  for item in vals:
    if item == 'true':
      res.append(1)
    elif item == 'false':
      res.append(0)
    elif item == 'unverified':
      res.append(2)
  return res

In [0]:
y_test = convert_test_veracity(test_y)

In [0]:
import numpy as np

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [0]:
documents = final_test_df['source'].values

In [0]:
vectorizer = CountVectorizer(max_features=10, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [0]:
len(X[0])

10

In [0]:
thm = np.array(has_media)
thm.resize(56, 1)
tur = np.array(upvote_ratio)
tur.resize(56, 1)
tfc = np.array(followers_count)
tfc.resize(56, 1)
tiv = np.array(is_verified)
tiv.resize(56, 1)

In [0]:
X=np.append(X, tur, axis=1)
X=np.append(X, thm, axis=1)
X=np.append(X, tfc, axis=1)
X=np.append(X, tiv, axis=1)

In [0]:
X_test = X

## Train Dataset Prep

In [0]:
def _finditem(obj, key):
    if key in obj: return obj[key]
    for k, v in obj.items():
        if isinstance(v,dict):
            item = _finditem(v, key)
            if item is not None:
                return item

In [0]:
is_verified = []
for item in x_tot:
  if _finditem(item,'verified'):
    is_verified.append(1)
  else:
    is_verified.append(0)

In [0]:
has_media = []
for item in x_tot:
  if (_finditem(item,'media')) is not None:
    has_media.append(1)
  else:
    has_media.append(0)

In [0]:
followers_count = []
for item in x_tot:
  followers_count.append(_finditem(item,'followers_count'))

In [0]:
upvote = []
for i in range(len(x_tot)):
  source_res = []
  for j in range(len(x_tot[i]['replies'])):
    source_res.append(x_tot[i]['replies'][j]['label'])
  upvote.append(source_res)

In [0]:
upvote_ratio = []
for item in upvote:
  support = item.count('support')
  upvote_ratio.append(support/len(item))

In [0]:
df = pd.DataFrame(columns=['source', 'veracity'])
count1=0
for i in range(len(x_tot)):
    df.loc[count1] = [x_tot[i]['source']['text'],(x_tot[i]['veracity'])]
    count1=count1+1
print(df.head())

                                              source veracity
0  France: 10 people dead after shooting at HQ of...     true
1  BREAKING: 10 reportedly shot dead at Paris HQ ...     true
2  BREAKING: At least 10 killed in shooting at Fr...     true
3  Eleven dead in shooting at Paris offices of sa...     true
4  BREAKING Charlie Hebdo latest: 11 dead 10 woun...     true


In [0]:
extra_features_df = pd.DataFrame(
    {
      'upvote_ratio':upvote_ratio,
      'followers_count':followers_count,
      'is_verified':is_verified,
      'has_media':has_media
       
    })

In [0]:
def convert_veracity(vals):
  res = []
  for item in vals.values:
    if item == 'true':
      res.append(1)
    elif item == 'false':
      res.append(0)
    elif item == 'unverified':
      res.append(2)
  return res

In [0]:
final_df= pd.DataFrame(
    {
      'source':df['source'].values,
      'upvote_ratio':upvote_ratio,
      'followers_count':followers_count,
      'is_verified':is_verified,
      'has_media':has_media,
      'veracity':convert_veracity(df['veracity'])
    })

In [0]:
final_df.head()

,source,upvote_ratio,followers_count,is_verified,has_media,veracity
0,France: 10 people dead after shooting at HQ of...,0.000000,129573,1,1,1
1,BREAKING: 10 reportedly shot dead at Paris HQ ...,0.666667,972167,1,0,1
2,BREAKING: At least 10 killed in shooting at Fr...,0.318182,3029912,1,0,1
3,Eleven dead in shooting at Paris offices of sa...,0.050000,3091451,1,0,1
4,BREAKING Charlie Hebdo latest: 11 dead 10 woun...,0.000000,973212,1,1,1


In [0]:
# final_df.to_csv('final_df.csv', sep=',')

In [0]:
documents = final_df['source'].values

In [0]:
vectorizer = CountVectorizer(max_features=10, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [0]:
hm = np.array(has_media)
hm.resize(325, 1)
ur = np.array(upvote_ratio)
ur.resize(325, 1)
fc = np.array(followers_count)
fc.resize(325, 1)
iv = np.array(is_verified)
iv.resize(325, 1)

In [0]:
fdf = final_df.copy()
ver = fdf['veracity'].values
ver.resize(325,1)

In [0]:
X.shape

(325, 10)

In [0]:
X=np.append(X, ur, axis=1)
X=np.append(X, hm, axis=1)
X=np.append(X, fc, axis=1)
X=np.append(X, iv, axis=1)

In [0]:
X_train = X

In [0]:
y_train = list(final_df['veracity'].values)

## simple machine learning models for Task B

In [0]:
# from sklearn.feature_extraction.text import TfidfTransformer
# tfidfconverter = TfidfTransformer()
# X = tfidfconverter.fit_transform(X).toarray()

## Data Prep

In [0]:
from sklearn.model_selection import train_test_split
y = final_df['veracity'].values

In [0]:
len(y)

325

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=20)

## Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=0)

classifier.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
y_pred = classifier.predict(X_test)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 3  3  2]
 [ 5 24  2]
 [ 3  8 15]]
              precision    recall  f1-score   support

           0       0.27      0.38      0.32         8
           1       0.69      0.77      0.73        31
           2       0.79      0.58      0.67        26

    accuracy                           0.65        65
   macro avg       0.58      0.58      0.57        65
weighted avg       0.68      0.65      0.65        65

0.6461538461538462


## Decision Tree

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [0]:
y_pred = clf.predict(X_test)

In [0]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 3  3  2]
 [ 6 18  7]
 [ 4  5 17]]
              precision    recall  f1-score   support

           0       0.23      0.38      0.29         8
           1       0.69      0.58      0.63        31
           2       0.65      0.65      0.65        26

    accuracy                           0.58        65
   macro avg       0.53      0.54      0.52        65
weighted avg       0.62      0.58      0.60        65

0.5846153846153846


## MLP Classifier

In [0]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, activation='relu',
                    hidden_layer_sizes=(10, 10), random_state=1)

clf.fit(X_train, y_train)                         


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 10), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
y_pred = clf.predict(X_test)

In [0]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 0  8  0]
 [ 0 31  0]
 [ 0 26  0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.48      1.00      0.65        31
           2       0.00      0.00      0.00        26

    accuracy                           0.48        65
   macro avg       0.16      0.33      0.22        65
weighted avg       0.23      0.48      0.31        65

0.47692307692307695


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Naive Bayes

In [0]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
y_pred = clf.predict(X_test)

In [0]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 0  1  7]
 [ 0  6 25]
 [ 0  6 20]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.46      0.19      0.27        31
           2       0.38      0.77      0.51        26

    accuracy                           0.40        65
   macro avg       0.28      0.32      0.26        65
weighted avg       0.37      0.40      0.34        65

0.4


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Gradient Boosted Trees

In [0]:
from sklearn import ensemble

In [0]:
clf = ensemble.GradientBoostingClassifier()
clf.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
y_pred = clf.predict(X_test)

In [0]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 2  4  2]
 [ 8 21  2]
 [ 2  8 16]]
              precision    recall  f1-score   support

           0       0.17      0.25      0.20         8
           1       0.64      0.68      0.66        31
           2       0.80      0.62      0.70        26

    accuracy                           0.60        65
   macro avg       0.53      0.51      0.52        65
weighted avg       0.64      0.60      0.62        65

0.6


## LSTM

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import KFold
tokenizer = Tokenizer(num_words=50, lower=True)
# print(documents[0])
seq=tokenizer.texts_to_sequences(final_df['source'])

X_lstm = pad_sequences(seq, maxlen=100)
# print(X_lstm)
X_lstm=np.append(X_lstm, ur, axis=1)
X_lstm=np.append(X_lstm, hm, axis=1)
X_lstm=np.append(X_lstm, fc, axis=1)
X_lstm=np.append(X_lstm, iv, axis=1)
print(X_lstm.shape)
n_split=5
cvscores = []
y_lstm=pd.get_dummies(final_df['veracity']).values
epochs = 10
batch_size = 32
 
for train_index, test_index in KFold(n_split).split(X_lstm):
  x_train,x_test=X_lstm[train_index],X_lstm[test_index]
  y_train,y_test=y_lstm[train_index],y_lstm[test_index]
  model = Sequential()
  model.add(Embedding(200, 100, input_length=X_lstm.shape[1]))
  model.add(SpatialDropout1D(0.2))
  model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
  scores = model.evaluate(x_test,y_test, verbose=0)
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  cvscores.append(scores[1] * 100)

ValueError: ignored

In [0]:
tokenizer = Tokenizer(num_words=50, lower=True)
seq=tokenizer.texts_to_sequences(final_test_df['source'])

X_lstm_test = pad_sequences(seq, maxlen=100)
# print(X_lstm)
X_lstm_test=np.append(X_lstm_test, tur, axis=1)
X_lstm_test=np.append(X_lstm_test, thm, axis=1)
X_lstm_test=np.append(X_lstm_test, tfc, axis=1)
X_lstm_test=np.append(X_lstm_test, tiv, axis=1)

In [0]:
y_lstm_test = pd.Series(y_test)

In [0]:
y_lstm_test = pd.get_dummies(y_lstm_test).values

In [0]:
scores = model.evaluate(X_lstm_test, y_lstm_test, verbose=0)

In [0]:
scores

[1.23715341091156, 0.19642857142857142]

In [0]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 104, 100)          20000     
_________________________________________________________________
spatial_dropout1d_11 (Spatia (None, 104, 100)          0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 303       
Total params: 100,703
Trainable params: 100,703
Non-trainable params: 0
_________________________________________________________________


<tf.Tensor 'dense_11/Softmax:0' shape=(?, 3) dtype=float32>

In [0]:
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [0]:
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 19.64%


In [0]:
from sklearn.metrics import classification_report


## Bert implementation for Task B

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 


--2019-11-11 11:03:31--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip.1’

uncased_L-12_H-768_ 100%[===================>] 388.84M   126MB/s    in 3.1s    

2019-11-11 11:03:34 (126 MB/s) - ‘uncased_L-12_H-768_A-12.zip.1’ saved [407727028/407727028]

--2019-11-11 11:03:35--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [t

In [0]:
import modeling
import optimization
import run_classifier
import tokenization
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

In [0]:
# from run_classifier import  convert

In [0]:
import zipfile
folder = 'model_folder'
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)

In [0]:
type(final_df['source'][0])

str

In [0]:
def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    print(type(lines))
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = str(line[0])
            text_b = str(line[1])
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    else:
        for line in lines:
            text_a = str(line[0])
            text_b = str(line[1])
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    return examples

In [0]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/output13'
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 5e-5
NUM_TRAIN_EPOCHS = 4.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 128
BOARD_SIZE=19
DROP_OUT=0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 100000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 100000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(3)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver =None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
         num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

<class 'numpy.ndarray'>
INFO:tensorflow:Using config: {'_model_dir': 'model_folder/output13', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f152b70a320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100000, num_shar

In [0]:
! rm -rf model_folder/output13

In [0]:
import datetime
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

Please wait...
INFO:tensorflow:Writing example 0 of 260
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train
INFO:tensorflow:tokens: [CLS] 0 . 0 [SEP] 0 . 0 [SEP]
INFO:tensorflow:input_ids: 101 1014 1012 1014 102 1014 1012 1014 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [0]:
X_test = final_test_df[['source','upvote_ratio']].values

In [0]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 50

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [0]:
predict_examples = create_examples(X_test, '')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)
len(X_test)
print(result)

<class 'numpy.ndarray'>
INFO:tensorflow:Writing example 0 of 65
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 
INFO:tensorflow:tokens: [CLS] 0 . 0 [SEP] 0 . 0 [SEP]
INFO:tensorflow:input_ids: 101 1014 1012 1014 102 1014 1012 1014 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [0]:
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))

{}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encode

In [0]:
print(len(preds),len(y_test))


56 65


In [0]:
tp=0
print(len(y_test),len(preds))
for i in range(len(y_test)):
  if y_test[i]==preds[i]:
    tp=tp+1
print("accuracy")
print(tp)

print(tp/len(preds))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,preds))

In [0]:
print(np.shape(y_test),np.shape(preds))
randlist=[]
for i in range(len(y_test)):
  if y_test[i]==preds[i] and y_test[i]==1:
    randlist.append(i)
print(len(randlist))
    

from sklearn.metrics import accuracy_score ,classification_report

print(classification_report(y_test,preds))

(56,) (56,)
4
              precision    recall  f1-score   support

           0       0.50      0.17      0.25        30
           1       0.29      0.18      0.22        22
           2       0.00      0.00      0.00         4

    accuracy                           0.16        56
   macro avg       0.26      0.12      0.16        56
weighted avg       0.38      0.16      0.22        56



In [0]:
def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    # print(type(lines))
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = str(line[0])
            text_b = None
            extra_features = line[1:]
            # print(label)
            label = str(label)
            examples.append(
              run_classifier.InputExampleExtraFeatures(guid=guid, text_a=text_a, text_b=text_b, label=label,extra_features=extra_features))
    else:
        for line in lines:
            text_a = str(line[0])
            text_b = None
            label = '0'
            extra_features = line[1:]
            examples.append(
              run_classifier.InputExampleExtraFeatures(guid=guid, text_a=text_a, text_b=text_b, label=label,extra_features=extra_features))
    return examples

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

In [0]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/output13'
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 5e-5
NUM_TRAIN_EPOCHS = 4.0
WARMUP_PROPORTION = 0.1
PREDICT_BATCH_SIZE = 32
MAX_SEQ_LENGTH = 128
BOARD_SIZE=19
DROP_OUT=0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 1000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(3)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver =None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
         num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder_extra_features(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator_extra_features = tf.contrib.tpu.TPUEstimator(
  use_tpu=False,
  model_fn=model_fn,
  config=run_config,
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': 'model_folder/output13', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f17b364a630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=

In [0]:
!rm -rf model_folder/output13

In [0]:
import datetime
print('Please wait...')
train_features = run_classifier.file_based_convert_examples_to_features_with_extra_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer, 'train_extra_features.TFRecord')
len(train_examples)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn_extra_features = run_classifier.file_based_input_fn_builder_extra_features(
    input_file='train_extra_features.TFRecord',
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

Please wait...
INFO:tensorflow:Writing example 0 of 325
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train
INFO:tensorflow:tokens: [CLS] 0 . 0 [SEP]
INFO:tensorflow:input_ids: 101 1014 1012 1014 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [0]:
estimator_extra_features.train(input_fn=train_input_fn_extra_features, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = extra_features, shape = (32, 4)
INFO:tensorflow:  name = input_ids, shape = (32, 128)
INFO:tensorflow:  name = input_mask, shape = (32, 128)
INFO:tensorflow:  name = is_real_example, shape = (32,)
INFO:tensorflow:  name = label_ids, shape = (32,)
INFO:tensorflow:  name = segment_ids, sh

InvalidArgumentError: ignored

In [0]:
# from run_classifier import con

In [0]:
predict_examples = create_examples(X_test, '')

In [0]:
predict_features = run_classifier.file_based_convert_examples_to_features_with_extra_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer, 'test_extra_features.TFRecord')

INFO:tensorflow:Writing example 0 of 56
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 
INFO:tensorflow:tokens: [CLS] 0 . 0 [SEP]
INFO:tensorflow:input_ids: 101 1014 1012 1014 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [0]:
predict_input_fn = run_classifier.file_based_input_fn_builder_extra_features(
    input_file='test_extra_features.TFRecord',
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator_extra_features.predict(input_fn=predict_input_fn)
print(result)

<generator object TPUEstimator.predict at 0x7f181e45bbf8>


In [0]:
preds = []
for prediction in result:
      print(prediction)
      preds.append(np.argmax(prediction['probabilities']))
print(len(preds),len(y_test))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = extra_features, shape = (?, 4)
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/laye

InvalidArgumentError: ignored

In [0]:
tp=0
for i in range(len(y_test)):
  if y_test[i]==preds[i]:
    tp=tp+1
print("accuracy")
print(tp/len(preds))

print(type(y_test))
print(type(preds))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test.tolist(),preds))

IndexError: ignored

In [0]:
print(np.shape(y_test),np.shape(preds))
randlist=[]
for i in range(len(y_test)):
  if y_test[i]==preds[i] and y_test[i]==1:
    randlist.append(i)
print(len(randlist))
    

from sklearn.metrics import accuracy_score ,classification_report
# print(preds)
# print("Accuracy of BERT is:",accuracy_score(y_test,preds))

print(classification_report(y_test.tolist(),preds))


(65,) (65,)
27
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.42      1.00      0.59        27
           2       0.00      0.00      0.00        21

    accuracy                           0.42        65
   macro avg       0.14      0.33      0.20        65
weighted avg       0.17      0.42      0.24        65



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
